# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""66768586033678…","""../../../../da…",1.2927e19,"""10477563659166…","""18236160113996…","""14006490190824…",false
"""13867084211674…","""../../../../da…",1.4856e19,"""14093914574360…","""18236160113996…","""13198167550845…",true
"""96938241932126…","""../../../../da…",5.6458e17,"""67107696723636…","""13005724425041…","""14006490190824…",false
"""42688921967326…","""../../../../da…",4.0723e18,"""14093914574360…","""51763933940117…","""14006490190824…",false
"""92792551704797…","""../../../../da…",1.4677e19,"""10477563659166…","""51763933940117…","""41745130970179…",true
"""17240665024304…","""../../../../da…",1.6330e19,"""67107696723636…","""77749608786652…",null,false
"""12100804368576…","""../../../../da…",8.7675e18,"""67107696723636…","""51763933940117…","""41745130970179…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""16629364776236…","""../../../../da…",1.2927e19,"""73123753938380…","""FirstMortgage3…","""15909574181061…","""Rejected"""
"""13337921724360…","""../../../../da…",1.4856e19,"""52109269766609…","""FirstMortgage3…","""33555343858338…","""Accepted"""
"""90282488322476…","""../../../../da…",5.6458e17,"""10196768547918…","""MoneyMarketSav…","""15909574181061…","""Rejected"""
"""10224622791397…","""../../../../da…",4.0723e18,"""52109269766609…","""BasicChecking""","""15909574181061…","""Rejected"""
"""13236942410032…","""../../../../da…",1.4677e19,"""73123753938380…","""BasicChecking""","""68250418393944…","""Accepted"""
"""64084331286494…","""../../../../da…",1.6330e19,"""10196768547918…","""UPlusFinPerson…",null,"""Rejected"""
"""61934305264499…","""../../../../da…",8.7675e18,"""10196768547918…","""BasicChecking""","""68250418393944…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""63912970497873…","""../../../../da…",1.2927e19,"""22825617778921…","""FirstMortgage3…","""16085051858691…","""Rejected"""
"""81797381650840…","""../../../../da…",1.4856e19,"""34539150814914…","""FirstMortgage3…","""16122510051445…","""Accepted"""
"""34309594204528…","""../../../../da…",5.6458e17,"""73624190329748…","""MoneyMarketSav…","""16085051858691…","""Rejected"""
"""24904224266797…","""../../../../da…",4.0723e18,"""34539150814914…","""BasicChecking""","""16085051858691…","""Rejected"""
"""17785129113049…","""../../../../da…",1.4677e19,"""22825617778921…","""BasicChecking""","""48274599490548…","""Accepted"""
"""62178883859845…","""../../../../da…",1.6330e19,"""73624190329748…","""UPlusFinPerson…",null,"""Rejected"""
"""59667973008392…","""../../../../da…",8.7675e18,"""73624190329748…","""BasicChecking""","""48274599490548…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""91079334692606…",1.2927e19,"""17182166433648…","""FirstMortgage3…","""33802603150933…","""Rejected"""
"""61789247635848…",1.4856e19,"""74844550794649…","""FirstMortgage3…","""15887277523769…","""Accepted"""
"""63065468735649…",5.6458e17,"""22065912682780…","""MoneyMarketSav…","""33802603150933…","""Rejected"""
"""88808001230762…",4.0723e18,"""74844550794649…","""BasicChecking""","""33802603150933…","""Rejected"""
"""13539371777842…",1.4677e19,"""17182166433648…","""BasicChecking""","""14275010561185…","""Accepted"""
"""62946551191443…",1.6330e19,"""22065912682780…","""UPlusFinPerson…",null,"""Rejected"""
"""65667149854768…",8.7675e18,"""22065912682780…","""BasicChecking""","""14275010561185…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'